In [6]:
%pip install keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.3 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
# Read libraries

# Data wrangling
import pandas as pd
import numpy as np
import re
from shapely import wkt
import fiona

# Visualization 
import missingno as msno
import matplotlib
import mapclassify
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go
import seaborn as sns
import geopandas as gpd


# Geostatistics
from mgwr.gwr import GWR
from mgwr.sel_bw import Sel_BW
import libpysal as ps
import matplotlib.pyplot as plt
import seaborn as sns
from pysal.lib import weights
from pysal.model import spreg
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.compat import lzip
from esda.moran import Moran_Local
from splot.esda import lisa_cluster
from pysal.lib import weights
from splot.libpysal import plot_spatial_weights
from spreg import OLS
from spreg import MoranRes
from spreg import ML_Lag
from spreg import ML_Error

# Machine learning 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.feature_selection import RFE
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier

import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'keras'

In [3]:
# Read the GeoParquet file without one-hot sampling
gdf_no_oh = gpd.read_parquet('gdf_no_ohe.parquet')

In [4]:
gdf_no_oh

,year,countries,region,pf_ss_disappearances_disap,pf_ss_disappearances_violent,pf_ss_disappearances_violent_data,pf_ss_disappearances_organized,pf_ss_disappearances_fatalities,pf_ss_disappearances_fatalities_data,pf_ss_disappearances_injuries,...,ef_regulation_business_compliance,ef_regulation_business,ef_regulation,country_code,geometry,globalNorth,ef_score,hf_score,pf_expression_bti,pf_rol_civil
0,2020.0,Albania,Eastern Europe,10.0,10.000000,0.0,7.500000,10.000000,0.0,10.000000,...,7.175250,6.698118,7.112958,ALB,"POLYGON ((20.07142 42.56091, 20.10208 42.53347...",1.0,0,0,0,0
1,2020.0,Algeria,Middle East & North Africa,10.0,9.687083,25.0,5.000000,10.000000,0.0,10.000000,...,7.029528,5.686753,5.778953,DZA,"POLYGON ((8.62203 36.94137, 8.63222 36.88194, ...",0.0,0,0,0,0
2,2020.0,Angola,Sub-Saharan Africa,10.0,9.582498,25.0,7.500000,9.736578,5.0,9.971733,...,6.782923,5.696942,6.227545,AGO,"MULTIPOLYGON (((23.98621 -10.87046, 23.98805 -...",0.0,0,0,0,0
3,2020.0,Argentina,Latin America & the Caribbean,5.0,10.000000,0.0,7.500000,9.925379,2.0,10.000000,...,6.508295,6.225232,5.490538,ARG,"MULTIPOLYGON (((-68.64312 -54.88861, -68.63723...",0.0,0,0,0,0
4,2020.0,Armenia,Caucasus & Central Asia,10.0,10.000000,0.0,7.500000,10.000000,0.0,10.000000,...,7.040738,7.124727,7.756333,ARM,"POLYGON ((46.54038 38.87559, 46.51639 38.87804...",1.0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3481,2000.0,Venezuela,Latin America & the Caribbean,10.0,10.000000,0.0,8.231718,10.000000,0.0,10.000000,...,0.315142,4.692328,5.531397,VEN,"MULTIPOLYGON (((-61.18507 8.49674, -61.19417 8...",0.0,0,0,0,1
3482,2000.0,Vietnam,South Asia,10.0,10.000000,0.0,7.964260,10.000000,0.0,10.000000,...,0.000000,3.861879,4.416768,VNM,"MULTIPOLYGON (((104.08288 10.36486, 104.08663 ...",0.0,0,0,0,1
3483,2000.0,Yemen,Middle East & North Africa,5.0,10.000000,0.0,7.044853,8.170079,19.0,7.932996,...,6.988352,5.658095,6.684552,YEM,"MULTIPOLYGON (((52.23416 12.20111, 52.27888 12...",0.0,1,1,0,1
3484,2000.0,Zambia,Sub-Saharan Africa,10.0,10.000000,0.0,7.552838,10.000000,0.0,9.827262,...,8.520369,8.020982,6.799421,ZMB,"POLYGON ((32.94040 -9.40508, 32.93944 -9.41583...",0.0,0,0,0,1
